In [1]:
import configparser
from datetime import datetime
import os
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, col
from pyspark.sql.functions import year, month, dayofmonth, hour, weekofyear, date_format


# Connect to AWS
config = configparser.ConfigParser()
config.read('dl_prod.cfg')

os.environ['AWS_ACCESS_KEY_ID']=config.get("CREDENTIALS", 'AWS_ACCESS_KEY_ID')
os.environ['AWS_SECRET_ACCESS_KEY']=config.get("CREDENTIALS", 'AWS_SECRET_ACCESS_KEY')

def create_spark_session():
    spark = SparkSession \
        .builder \
        .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:2.7.0") \
        .getOrCreate()
    return spark

In [2]:
spark = create_spark_session()
input_data = "s3a://udacity-dend/"
input_data = 's3://udacity-dend/'
input_data = 'data'
output_data="D://OneDrive - Watson + Holmes/WerkbestandenYannick/Werkmap_Python/Projecten/Studie/Udacity/projects_data_engineer/project4_Spark_datalakes/data"

# process_song_data(spark, input_data, output_data)

In [3]:
os.getcwd()

'D:\\OneDrive - Watson + Holmes\\WerkbestandenYannick\\Werkmap_Python\\Projecten\\Studie\\Udacity\\projects_data_engineer\\project4_Spark_datalakes'

In [4]:
# get filepath to song data file
song_data = f'{input_data}/song_data/*/*/*/*.json'
# song_data = f'{input_data}/song-data.zip'

# read song data file
df = spark.read.json(song_data)

# # extract columns to create songs table
# songs_table = 

# # write songs table to parquet files partitioned by year and artist
# songs_table

# # extract columns to create artists table
# artists_table = 

# # write artists table to parquet files
# artists_table

In [5]:
df.head()

Row(artist_id='ARDR4AC1187FB371A1', artist_latitude=None, artist_location='', artist_longitude=None, artist_name='Montserrat Caballé;Placido Domingo;Vicente Sardinero;Judith Blegen;Sherrill Milnes;Georg Solti', duration=511.16363, num_songs=1, song_id='SOBAYLL12A8C138AF9', title='Sono andati? Fingevo di dormire', year=0)

In [6]:
df_song = df.select(["song_id", "title", "artist_id", "year", "duration"]).drop_duplicates()

In [7]:
df_song.limit(1).show(5)

+------------------+-----------------+------------------+----+---------+
|           song_id|            title|         artist_id|year| duration|
+------------------+-----------------+------------------+----+---------+
|SOGOSOV12AF72A285E|¿Dónde va Chichi?|ARGUVEV1187B98BA17|1997|313.12934|
+------------------+-----------------+------------------+----+---------+



In [10]:
df_song.write.parquet(output_data + 'songs.parquet', partitionBy=('year', 'artist_id'), mode='overwrite')

Py4JJavaError: An error occurred while calling o128.parquet.
: org.apache.spark.SparkException: Job aborted.
	at org.apache.spark.sql.execution.datasources.FileFormatWriter$.write(FileFormatWriter.scala:198)
	at org.apache.spark.sql.execution.datasources.InsertIntoHadoopFsRelationCommand.run(InsertIntoHadoopFsRelationCommand.scala:159)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult$lzycompute(commands.scala:104)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult(commands.scala:102)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.doExecute(commands.scala:122)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$execute$1.apply(SparkPlan.scala:131)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$execute$1.apply(SparkPlan.scala:127)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$executeQuery$1.apply(SparkPlan.scala:155)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOpera

In [9]:
# df_song.write.parquet(f"/{output_data}/test", mode='overwrite', partitionBy=['year', 'artist_id'])#.format("parquet").saveAsTable("tab1")
# df_song.limit(1).write.mode(saveMode='Overwrite').format("parquet").save("tab1")
df_song.limit(1).write.parquet(f"{input}/test") #, mode='overwrite', partitionBy=['year', 'artist_id'])#.format("parquet").saveAsTable("tab1")


Py4JJavaError: An error occurred while calling o87.parquet.
: org.apache.spark.SparkException: Job aborted.
	at org.apache.spark.sql.execution.datasources.FileFormatWriter$.write(FileFormatWriter.scala:198)
	at org.apache.spark.sql.execution.datasources.InsertIntoHadoopFsRelationCommand.run(InsertIntoHadoopFsRelationCommand.scala:159)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult$lzycompute(commands.scala:104)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult(commands.scala:102)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.doExecute(commands.scala:122)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$execute$1.apply(SparkPlan.scala:131)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$execute$1.apply(SparkPlan.scala:127)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$executeQuery$1.apply(SparkPlan.scala:155)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperat